In [ ]:
# FOR WINDOWS (your env must be called project-venv; if you choose another name add it in .gitignore)
import subprocess

# Set the execution policy
subprocess.run(["Set-ExecutionPolicy", "RemoteSigned", "-Scope", "Process"], shell=True)

# Activate the virtual environment
subprocess.run(["cd", ".\\project-venv\\"], shell=True)
subprocess.run([".\\Scripts\\Activate.ps1"], shell=True)

# Install requirements
subprocess.run(["pip", "install", "-r", "../requirements.txt"], shell=True)

# **Emotion Recognition task**

**import packages**

In [1]:
import cv2
import os
import random
import numpy as np
from scipy.ndimage import gaussian_filter, map_coordinates
import torch
import torch.nn as nn
from sklearn.cluster import DBSCAN
from torch.utils.data import random_split, ConcatDataset
from torchvision.datasets import ImageFolder
from torchvision import transforms
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from torch.utils.data.sampler import SubsetRandomSampler
from torch.optim import SGD
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
from tqdm import tqdm
import dlib
from PIL import Image
import wandb

**wandb login**

In [2]:
wandb.login(key='99d71868d6134c943615e698d15b77af2dd5de4a')

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: bertagnoli (computer_visionistici). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\danie/.netrc


True

## **Paper implementation 1**
https://ieeexplore.ieee.org/abstract/document/9659697?casa_token=zDD7lwwOig8AAAAA:KcIHhupXAXgiaB_C7A0uNDB7ehrsWNyovQdgDu9LmnwToOGU6akB_gjWTy7JCf4UdKK03Is

https://github.com/serengil/deepface

### **Dataset Augmenting**

#### **Objective:**
The purpose of this Python script is to perform data augmentation on a dataset of images corresponding to different emotions. Data augmentation is a technique commonly used in computer vision tasks to increase the diversity of the training dataset, thereby enhancing the robustness and generalization capabilities of machine learning models.

#### **Dataset Structure:**
The original dataset is organized into folders, each representing a specific emotion (e.g., angry, disgust, fear, happy, neutral, sad, surprise). Each emotion folder contains a collection of images in formats such as JPEG and PNG.

#### **Transformation Techniques:**
The script employs various image transformation techniques to augment the dataset. These techniques include:

1. **Horizontal Flip:** Flips the image horizontally.
2. **Vertical Flip:** Flips the image vertically.
3. **Zoom:** Randomly zooms into or out of the image.
4. **Translation:** Shifts the image horizontally and vertically.
5. **Contrast and Brightness Adjustment:** Randomly adjusts the contrast and brightness of the image.
6. **Elastic Transformation:** Applies a non-linear elastic deformation to the image.

#### **Implementation:**
The script uses the OpenCV library for image processing. For each emotion category, it iterates through the images, applies the defined transformations using the `apply_transformations` function, and saves the augmented images in a new folder named with the emotion followed by "_augmented."

The `elastic_transform` function generates an elastic transformation by displacing pixels based on random elastic deformations. This adds a degree of distortion to the images, contributing to further variability.

#### **Output:**
The augmented images are saved in the corresponding emotion's augmented folder, with filenames indicating the applied augmentation technique. For example, an image originally named "example_image.jpg" may result in augmented images like "example_image_aug_0.jpg," "example_image_aug_1.jpg," etc.

In [ ]:
emotions = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']

# cycle through emotions
for emotion in emotions:
    # path of the folder containing the images
    folder_path = f"../Data/emotions_images/{emotion}"
    output_folder_path = f"../Data/emotions_aug_images/{emotion}_augmented"


    if not os.path.exists(output_folder_path):
        os.makedirs(output_folder_path)

    # list of images in the folder
    image_files = [f for f in os.listdir(folder_path) if f.endswith(('.jpg', '.jpeg', '.png'))]

    # define transformations inside the apply_transformations function
    def apply_transformations(image):
        # horizontal_flip
        flipped_horizontal = cv2.flip(image, 1)

        # vertical flip
        flipped_vertical = cv2.flip(image, 0)

        # Zoom
        zoom_factor = random.uniform(0.8, 1.2)
        height, width = image.shape[:2]
        zoomed_image = cv2.resize(image, (int(width * zoom_factor), int(height * zoom_factor)))

        # translation
        tx = random.randint(-10, 10)
        ty = random.randint(-10, 10)
        translation_matrix = np.float32([[1, 0, tx], [0, 1, ty]])
        translated_image = cv2.warpAffine(image, translation_matrix, (width, height))

        # contrast and brightness control
        alpha = random.uniform(0.8, 1.2)
        beta = random.randint(-35, 35)
        adjusted_image = cv2.convertScaleAbs(image, alpha=alpha, beta=beta)

        # elastic transformation
        elastic_image = elastic_transform(image, alpha=random.randint(6, 14), sigma=random.uniform(1.1, 2.0))

        return [image, translated_image, flipped_horizontal, zoomed_image, adjusted_image, elastic_image] #forse togliere flipped vertical

    def elastic_transform(image, alpha, sigma):
        random_state = np.random.RandomState(None)
        shape = image.shape
        dx = gaussian_filter((random_state.rand(*shape) * 2 - 1), sigma, mode="constant", cval=0) * alpha
        dy = gaussian_filter((random_state.rand(*shape) * 2 - 1), sigma, mode="constant", cval=0) * alpha
        dz = np.zeros_like(dx)

        x, y, z = np.meshgrid(np.arange(shape[1]), np.arange(shape[0]), np.arange(shape[2]))
        indices = np.reshape(y + dy, (-1, 1)), np.reshape(x + dx, (-1, 1)), np.reshape(z + dz, (-1, 1))

        distorted_image = map_coordinates(image, indices, order=1, mode='reflect')
        distorted_image = distorted_image.reshape(image.shape)

        return distorted_image

    # apply data augmentation
    for image_file in image_files:
        image_path = os.path.join(folder_path, image_file)
        image = cv2.imread(image_path)

        augmented_images = apply_transformations(image)

        # save new images
        base_name = os.path.splitext(image_file)[0]
        for i, augmented_image in enumerate(augmented_images):
            output_file_path = os.path.join(output_folder_path, f"{base_name}_aug_{i}.jpg")
            cv2.imwrite(output_file_path, augmented_image)

### **CNN Architecture: EmotionCNN**

#### **Overview:**
The `EmotionCNN` model is designed for emotion classification using a Convolutional Neural Network (CNN). Below is an overview of its architecture:

#### **Architecture Layers:**

1. **Input Layer:**
   - Input channels: 1 (grayscale images)
   - Input size: Variable (depends on the input image size)

2. **Convolutional Layer 1:**
   - Convolutional operation: 64 filters of size 5x5
   - Activation function: ReLU
   - Max pooling: 5x5 kernel with stride 2

3. **Convolutional Layers 2a and 2b:**
   - Convolutional operation: 64 filters of size 3x3
   - Activation functions: ReLU for both layers
   - Average pooling: 3x3 kernel with stride 2

4. **Convolutional Layers 3a and 3b:**
   - Convolutional operation: 128 filters of size 3x3
   - Activation functions: ReLU for both layers
   - Average pooling: 3x3 kernel with stride 2

5. **Fully Connected (FC) Layer 1:**
   - Units: 1024
   - Activation function: ReLU
   - Dropout: 20% dropout rate

6. **Fully Connected (FC) Layer 2:**
   - Units: 1024
   - Activation function: ReLU
   - Dropout: 20% dropout rate

7. **Output Layer:**
   - Units: Variable (depends on the number of emotion classes)
   - Activation function: Softmax

#### **Additional Information:**
- The architecture is designed for grayscale images (1 channel).
- The network utilizes ReLU activation functions after convolutional and fully connected layers.
- Max pooling and average pooling layers are employed for down-sampling.
- Dropout layers (20% dropout rate) are included for regularization.
- The final layer uses the softmax activation for multiclass classification.

In [3]:
class EmotionCNN(nn.Module):
    def __init__(self, num_classes=7):
        super(EmotionCNN, self).__init__()
        
        self.conv1 = nn.Conv2d(1, 64, kernel_size=5, stride=1, padding=0)
        self.relu1 = nn.ReLU()
        self.maxpool1 = nn.MaxPool2d(kernel_size=5, stride=2)
        
        self.conv2a = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1)
        self.relu2a = nn.ReLU()
        self.conv2b = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1)
        self.relu2b = nn.ReLU()
        self.avgpool2 = nn.AvgPool2d(kernel_size=3, stride=2)
        
        self.conv3a = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.relu3a = nn.ReLU()
        self.conv3b = nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1)
        self.relu3b = nn.ReLU()
        self.avgpool3 = nn.AvgPool2d(kernel_size=3, stride=2)
        
        # verify the output size of conv2 and conv3
        self.dummy_input = torch.randn(1, 1, 48, 48)
        self.dummy_output_size = self._get_conv_output_size(self.dummy_input)
        
        # update fc1 units based on feature map size
        self.fc1 = nn.Linear(self.dummy_output_size, 1024)
        self.relu_fc1 = nn.ReLU()
        self.dropout1 = nn.Dropout(0.2)
        
        self.fc2 = nn.Linear(1024, 1024)
        self.relu_fc2 = nn.ReLU()
        self.dropout2 = nn.Dropout(0.2)
        
        self.fc3 = nn.Linear(1024, num_classes)
        self.softmax = nn.Softmax(dim=1)

    def _get_conv_output_size(self, input_tensor):
        x = self.maxpool1(self.relu1(self.conv1(input_tensor)))
        x = self.relu2a(self.conv2a(x))
        x = self.relu2b(self.conv2b(x))
        x = self.avgpool2(x)
        x = self.relu3a(self.conv3a(x))
        x = self.relu3b(self.conv3b(x))
        x = self.avgpool3(x)
        return x.view(x.size(0), -1).size(1)

    def forward(self, x):
        x = self.maxpool1(self.relu1(self.conv1(x)))
        x = self.relu2a(self.conv2a(x))
        x = self.relu2b(self.conv2b(x))
        x = self.avgpool2(x)
        x = self.relu3a(self.conv3a(x))
        x = self.relu3b(self.conv3b(x))
        x = self.avgpool3(x)
        x = x.view(x.size(0), -1)
        x = self.dropout1(self.relu_fc1(self.fc1(x)))
        x = self.dropout2(self.relu_fc2(self.fc2(x)))
        x = self.softmax(self.fc3(x))
        return x

### **Delete Outliers with DBSCAN**

#### **Objective:**
The Python script is designed to identify and remove outliers from a collection of images using the Density-Based Spatial Clustering of Applications with Noise (DBSCAN) algorithm. The primary goal is to filter out images with unusual pixel statistics, specifically focusing on standard deviation.

#### **Functions:**

1. **`calculate_pixel_std(image_path):`**
   - **Input:** Path to an image file.
   - **Output:** Returns the standard deviation of pixel values in the grayscale image.

2. **`remove_outliers_dbscan(folder_path, eps, min_samples):`**
   - **Input:**
     - `folder_path`: Path to the folder containing images.
     - `eps`: DBSCAN search radius parameter.
     - `min_samples`: Minimum number of samples required for a cluster in DBSCAN.
   - **Action:**
     - Calculates the pixel standard deviation for each image in the folder.
     - Applies DBSCAN to cluster images based on pixel standard deviation.
     - Removes outliers (images with label -1) by deleting them from the folder.
   - **Output:** Prints the number of outliers removed.

#### **Usage:**

1. Set the DBSCAN configuration parameters:
   - `dbscan_eps`: DBSCAN search radius.
   - `dbscan_min_samples`: Minimum number of samples required for a cluster.

2. Provide the path to the folder containing emotion-specific image folders (`emotions_folder_path`).

3. Iterate through each emotion folder:
   - If the emotion is 'disgust,' use different DBSCAN parameters.
   - Apply `remove_outliers_dbscan` to delete outliers in each emotion folder.

4. The script prints the number of outliers removed for each emotion.

In [ ]:
def calculate_pixel_std(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    return np.std(image)

def remove_outliers_dbscan(folder_path, eps, min_samples):
    images = []
    for filename in os.listdir(folder_path):
        image_path = os.path.join(folder_path, filename)
        pixel_std = calculate_pixel_std(image_path)
        images.append([pixel_std])

    images = np.array(images)

    # dbscan to identify outliers
    dbscan = DBSCAN(eps=eps, min_samples=min_samples)
    labels = dbscan.fit_predict(images)

    # remove outliers
    counter = 0
    for i, (label, image) in enumerate(zip(labels, os.listdir(folder_path))):
        if label == -1:  
            image_path = os.path.join(folder_path, image)
            os.remove(image_path)
            counter += 1
    print(counter)

# DBSCAN configuration
dbscan_eps = 0.5  # search radius
dbscan_min_samples = 10  # minimum number of samples required for a cluster

emotions_folder_path = "../Data/emotions_aug_images"


for emotion in os.listdir(emotions_folder_path):
    emotion_folder_path = os.path.join(emotions_folder_path, emotion)
    print(emotion)
    
    if emotion == 'disgust':
        tmp_folder_path = emotion_folder_path 
        remove_outliers_dbscan(tmp_folder_path, 0.5, 10)
    else:
        remove_outliers_dbscan(emotion_folder_path, dbscan_eps, dbscan_min_samples)

Print the path and the number of the outliers per class

In [ ]:
def calculate_pixel_std(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    return np.std(image)

def get_outliers_dbscan(folder_path, eps, min_samples):
    images = []
    image_paths = []

    for filename in os.listdir(folder_path):
        image_path = os.path.join(folder_path, filename)
        pixel_std = calculate_pixel_std(image_path)
        images.append([pixel_std])
        image_paths.append(image_path)

    images = np.array(images)

    # dbscan to identify outliers
    dbscan = DBSCAN(eps=eps, min_samples=min_samples)
    labels = dbscan.fit_predict(images)

    # collect outlier paths and count for each label
    outlier_paths_by_label = {}
    for label, image_path in zip(labels, image_paths):
        if label == -1:
            if label not in outlier_paths_by_label:
                outlier_paths_by_label[label] = []
            outlier_paths_by_label[label].append(image_path)

    return outlier_paths_by_label

# Configurazione DBSCAN
dbscan_eps = 0.5  # Raggio di ricerca
dbscan_min_samples = 10  # Numero minimo di campioni in un cluster

emotions_folder_path = "../Data/emotions_aug_images"

for emotion in os.listdir(emotions_folder_path):
    emotion_folder_path = os.path.join(emotions_folder_path, emotion)
    print(emotion)

    if emotion == 'disgust':
        tmp_folder_path = emotion_folder_path 
        outliers = get_outliers_dbscan(tmp_folder_path, 0.5, 10)
    else:
        outliers = get_outliers_dbscan(emotion_folder_path, dbscan_eps, dbscan_min_samples)

    # Stampa i percorsi degli outliers per ogni label
    for label, outlier_paths in outliers.items():
        print(f"Label {label}: {len(outlier_paths)} outliers")
        for path in outlier_paths:
            print(path)

### **Emotion Recognition Training and Evaluation**

This code performs emotion recognition using a Convolutional Neural Network (CNN). Key components include:

- **Emotion Label Mapping**: Emotions are mapped to numerical values for training and evaluation.
- **Model Definition and Setup**: A CNN model, along with the criterion, optimizer, and scheduler, is defined.
- **Metrics Calculation**: The `calculate_metrics_per_class` function computes accuracy, precision, recall, F1 score, and support for each class.
- **Training Function**: The `train_epoch` function trains the model for one epoch, logging loss, accuracy, and metrics per class.
- **Evaluation Function**: The `evaluate` function assesses the model on the validation set, calculating loss, accuracy, and metrics per class.
- **Test Function**: The `test` function evaluates the model on the test set, providing loss, accuracy, and metrics per class.
- **Training and Evaluation Process**: The `train_and_evaluate` function orchestrates the entire process. It initializes WandB, sets up data transformations, creates weighted random samplers for handling class imbalances, and conducts training and validation. The best model is saved, and evaluation metrics are logged.
- **Hyperparameter Sweep Configuration**: The code supports hyperparameter optimization using WandB sweeps. Parameters include the number of instances for over/under-sampling, batch size, and epochs.

#### Data Preprocessing

Two datasets are utilized with different transformations, addressing grayscale, resizing, and normalization. Weighted random samplers handle class imbalances.

#### Training Process

The training process involves iterating through epochs, training on the specified dataset, and evaluating on validation data. Early stopping is implemented to prevent overfitting.

#### Logging and Visualization

WandB is integrated for experiment tracking. Metrics such as loss, accuracy, and metrics per class are logged during training and evaluation.

In [4]:
your_label_mapping = {0: 'Angry', 1: 'Disgust', 2: 'Fear', 3: 'Happy', 4: 'Neutral', 5: 'Sad', 6: 'Surprise'}

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


def calculate_metrics_per_class(true_labels, predicted_labels, label_mapping):
    unique_labels = list(label_mapping.keys())
    precision, recall, f1, support = precision_recall_fscore_support(true_labels, predicted_labels, labels=unique_labels)
    accuracy = accuracy_score(true_labels, predicted_labels)
    
    metrics_per_class = {}
    for i, idx in enumerate(unique_labels):
        metrics_per_class[idx] = {
            'precision': precision[i],
            'recall': recall[i],
            'f1': f1[i],
            'support': support[i]
        }

    return accuracy, metrics_per_class

# function for training
def train_epoch(model, train_loader, criterion, optimizer, device, label_mapping):
    model.train()
    running_loss = 0.0
    true_labels = []
    predicted_labels = []

    for inputs, labels in tqdm(train_loader, desc='Training', leave=False):
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        _, preds = torch.max(outputs, 1)
        true_labels.extend(labels.cpu().numpy())
        predicted_labels.extend(preds.cpu().numpy())

    average_loss = running_loss / len(train_loader)
    accuracy, metrics_per_class = calculate_metrics_per_class(true_labels, predicted_labels, label_mapping)

    return average_loss, accuracy, metrics_per_class

# function for evaluation
def evaluate(model, val_loader, criterion, device, label_mapping):
    model.eval()
    running_loss = 0.0
    true_labels = []
    predicted_labels = []

    with torch.no_grad():
        for inputs, labels in tqdm(val_loader, desc='Validation', leave=False):
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item()

            _, preds = torch.max(outputs, 1)
            true_labels.extend(labels.cpu().numpy())
            predicted_labels.extend(preds.cpu().numpy())

    average_loss = running_loss / len(val_loader)
    accuracy, metrics_per_class = calculate_metrics_per_class(true_labels, predicted_labels, label_mapping)

    return average_loss, accuracy, metrics_per_class

# Funzione per il test
def test(model, test_loader, criterion, device, label_mapping):
    model.eval()
    running_loss = 0.0
    true_labels = []
    predicted_labels = []

    with torch.no_grad():
        for inputs, labels in tqdm(test_loader, desc='Testing', leave=False):
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss.item()

            _, preds = torch.max(outputs, 1)
            true_labels.extend(labels.cpu().numpy())
            predicted_labels.extend(preds.cpu().numpy())

    average_loss = running_loss / len(test_loader)
    accuracy, metrics_per_class = calculate_metrics_per_class(true_labels, predicted_labels, label_mapping)

    return average_loss, accuracy, metrics_per_class


def train_and_evaluate():
    wandb.init(project="Emotion_Recognition")

    number_instances_over_under_sampling_ = wandb.config.get('number_instances_over_under_sampling_')
    batch_size_ = wandb.config.get('batch_size_')
    epochs_ = wandb.config.get('epochs_')

    # definition of the model, criterion, optimizer and scheduler
    net = EmotionCNN().to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = SGD(net.parameters(), lr=0.01, momentum=0.9, nesterov=True, weight_decay=0.0001)
    scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.75, patience=3, verbose=True)

        # transformation definition
    transform = transforms.Compose([
        transforms.Grayscale(num_output_channels=1),
        transforms.Resize((48, 48)),
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])

    dataset_root = '../Data/test_images_emotion'

    # create an instance of ImageFolder with the transformations
    dataset = ImageFolder(root=dataset_root, transform=transform)

    # seed = 42
    torch.manual_seed(42)

    # extract the labels and the indices of the dataset
    labels = [label for _, label in dataset.imgs]

    # convert the list into a tensor
    labels = torch.tensor(labels)

    # calculate the number of instances for each class
    counts = torch.bincount(labels)

    # calculate the weights for each class
    weights = 1.0 / counts.float()

    # create a weight vector for each index in the dataset
    sample_weights = weights[labels]

    # set the number of samples for the train set and the test set
    train_size = (number_instances_over_under_sampling_/10) * 7 * 0.1
    val_size = (number_instances_over_under_sampling_/10) * 7 * 0.1
    test_size = (number_instances_over_under_sampling_/10) * 7 * 0.8

    # crea un sampler per il train set and one for the test set
    train_sampler = torch.utils.data.WeightedRandomSampler(sample_weights, int(train_size))
    val_sampler = torch.utils.data.WeightedRandomSampler(sample_weights, int(val_size))
    test_sampler = torch.utils.data.WeightedRandomSampler(sample_weights, int(test_size))

    # create a dataloader for the train set and the test set with the corresponding samplers
    train_loader_ = DataLoader(dataset, batch_size=batch_size_, sampler=train_sampler, num_workers=4)
    val_loader_ = DataLoader(dataset, batch_size=batch_size_, sampler=val_sampler, num_workers=4)
    test_loader_ = DataLoader(dataset, batch_size=batch_size_, sampler=test_sampler, num_workers=4)

            # transformation definition
    transform = transforms.Compose([
        transforms.Grayscale(num_output_channels=1),
        transforms.Resize((48, 48)),
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,))
    ])

    dataset_root = '../Data/emotions_aug_images'

    # create an instance of ImageFolder with the transformations
    dataset = ImageFolder(root=dataset_root, transform=transform)

    # seed = 42
    torch.manual_seed(42)

    # extract the labels and the indices of the dataset
    labels = [label for _, label in dataset.imgs]

    # convert the list into a tensor
    labels = torch.tensor(labels)

    # calculate the number of instances for each class
    counts = torch.bincount(labels)

    # calculate the weights for each class
    weights = 1.0 / counts.float()

    # create a weight vector for each index in the dataset
    sample_weights = weights[labels]

    # set the number of samples for the train set and the test set
    train_size = number_instances_over_under_sampling_ * 7 * 0.8
    val_size = number_instances_over_under_sampling_ * 7 * 0.1
    test_size = number_instances_over_under_sampling_ * 7 * 0.1

    # crea un sampler per il train set and one for the test set
    train_sampler = torch.utils.data.WeightedRandomSampler(sample_weights, int(train_size))
    val_sampler = torch.utils.data.WeightedRandomSampler(sample_weights, int(val_size))
    test_sampler = torch.utils.data.WeightedRandomSampler(sample_weights, int(test_size))

    # create a dataloader for the train set and the test set with the corresponding samplers
    train_loader = DataLoader(dataset, batch_size=batch_size_, sampler=train_sampler, num_workers=4)
    val_loader = DataLoader(dataset, batch_size=batch_size_, sampler=val_sampler, num_workers=4)
    test_loader = DataLoader(dataset, batch_size=batch_size_, sampler=test_sampler, num_workers=4)

    your_label_mapping = {0: 'Angry', 1: 'Disgust', 2: 'Fear', 3: 'Happy', 4: 'Neutral', 5: 'Sad', 6: 'Surprise'}

    # Settings
    num_epochs = epochs_
    early_stopping_patience = 3  # numbers of epochs with no improvement after which training will be stopped (early stopping)
    best_accuracy = 0.0
    best_epoch = 0
    no_improvement_count = 0

    train_loss = 0.0
    train_accuracy = 0.0
    train_metrics_per_class = {}

    val_loss = 0.0
    val_accuracy = 0.0
    val_metrics_per_class = {}

    # Training cycle
    for epoch in range(num_epochs):
        # Training
        train_loss, train_accuracy, train_metrics_per_class = train_epoch(net, train_loader, criterion, optimizer, device, your_label_mapping)

        # Validation
        val_loss, val_accuracy, val_metrics_per_class = evaluate(net, val_loader, criterion, device, your_label_mapping)

        # Scheduler step based on validation accuracy
        scheduler.step(val_accuracy)

        # Saving the model if the current accuracy is better than the best
        if val_accuracy > best_accuracy:
            best_accuracy = val_accuracy
            best_epoch = epoch
            torch.save(net.state_dict(), '../Models/emotion_recognition_model.pth')
            no_improvement_count = 0
        else:
            no_improvement_count += 1

        # Print epoch statistics
        print(f'Epoch {epoch + 1}/{num_epochs} => '
            f'Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}')

        # Print metrics per class
        for idx, label in your_label_mapping.items():
            print(f'{label}: Train Precision: {train_metrics_per_class[idx]["precision"]:.4f}, Train Recall: {train_metrics_per_class[idx]["recall"]:.4f}, Train F1: {train_metrics_per_class[idx]["f1"]:.4f}, Train Support: {train_metrics_per_class[idx]["support"]}')

        print(f'Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}')
        
        for idx, label in your_label_mapping.items():
            print(f'{label}: Validation Precision: {val_metrics_per_class[idx]["precision"]:.4f}, Validation Recall: {val_metrics_per_class[idx]["recall"]:.4f}, Validation F1: {val_metrics_per_class[idx]["f1"]:.4f}, Validation Support: {val_metrics_per_class[idx]["support"]}')

        if no_improvement_count >= early_stopping_patience:
            print(f'Early stopping at epoch {epoch + 1} as there is no improvement in validation accuracy for {early_stopping_patience} consecutive epochs.')
            break

    print(f'Best model achieved at epoch {best_epoch + 1} with accuracy {best_accuracy:.4f}')

    best_model = EmotionCNN()
    best_model.load_state_dict(torch.load('../Models/emotion_recognition_model.pth'))
    best_model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = SGD(net.parameters(), lr=0.01, momentum=0.9, nesterov=True, weight_decay=0.0001)
    scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.75, patience=3, verbose=True)
    test_loss, test_accuracy, test_metrics_per_class = test(best_model, test_loader, criterion, device, your_label_mapping)

    best_model = EmotionCNN()
    best_model.load_state_dict(torch.load('../Models/emotion_recognition_model.pth'))
    best_model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = SGD(net.parameters(), lr=0.01, momentum=0.9, nesterov=True, weight_decay=0.0001)
    scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.75, patience=3, verbose=True)
    test_loss_, test_accuracy_, test_metrics_per_class_ = test(best_model, test_loader_, criterion, device, your_label_mapping)

    print("TEST ACCURACY_: ", test_accuracy_)
    print("TEST ACCURACY: ", test_accuracy)

    # log the best model
    wandb.log({"train_loss": train_loss, "train_accuracy": train_accuracy, "val_loss": val_loss, "val_accuracy": val_accuracy, "test_accuracy": test_accuracy, "test_accuracy_": test_accuracy_})

    # log metrics per class
    for idx, label in your_label_mapping.items():
        wandb.log({
            f'train_precision_{label}': train_metrics_per_class[idx]["precision"],
            f'train_recall_{label}': train_metrics_per_class[idx]["recall"],
            f'train_f1_{label}': train_metrics_per_class[idx]["f1"],
            f'train_support_{label}': train_metrics_per_class[idx]["support"],
        })

    for idx, label in your_label_mapping.items():
        wandb.log({
            f'val_precision_{label}': val_metrics_per_class[idx]["precision"],
            f'val_recall_{label}': val_metrics_per_class[idx]["recall"],
            f'val_f1_{label}': val_metrics_per_class[idx]["f1"],
            f'val_support_{label}': val_metrics_per_class[idx]["support"],
        })


# sweep configuration
sweep_config = {
    "method": "grid",
    "parameters": {
        "number_instances_over_under_sampling_": {"values": [20000, 30000]},
        "batch_size_": {"values": [32, 48, 64]},
        "epochs_": {"values": [19, 23, 28]},
    }
}

# sweep inizialization
sweep_id = wandb.sweep(sweep=sweep_config, project="Emotion_Recognition")

# sweep execution

wandb.agent(sweep_id, function=train_and_evaluate)

Create sweep with ID: r4o87j67
Sweep URL: https://wandb.ai/computer_visionistici/Emotion_Recognition/sweeps/r4o87j67


wandb: Agent Starting Run: f685fpns with config:
wandb: 	batch_size_: 32
wandb: 	epochs_: 19
wandb: 	number_instances_over_under_sampling_: 20000
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


c:\Users\danie\Desktop\ComputerVisionProject\project-venv\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\danie\Desktop\ComputerVisionProject\project-venv\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/19 => Train Loss: 1.9446, Train Accuracy: 0.1486
Angry: Train Precision: 0.1770, Train Recall: 0.0441, Train F1: 0.0706, Train Support: 16163
Disgust: Train Precision: 0.1423, Train Recall: 0.0597, Train F1: 0.0841, Train Support: 15870
Fear: Train Precision: 0.0714, Train Recall: 0.0001, Train F1: 0.0001, Train Support: 16006
Happy: Train Precision: 0.0000, Train Recall: 0.0000, Train F1: 0.0000, Train Support: 15886
Neutral: Train Precision: 0.1190, Train Recall: 0.0003, Train F1: 0.0006, Train Support: 15980
Sad: Train Precision: 0.0000, Train Recall: 0.0000, Train F1: 0.0000, Train Support: 15859
Surprise: Train Precision: 0.1479, Train Recall: 0.9222, Train F1: 0.2549, Train Support: 16236
Validation Loss: 1.9359, Validation Accuracy: 0.1744
Angry: Validation Precision: 0.1659, Validation Recall: 0.4031, Validation F1: 0.2350, Validation Support: 2017
Disgust: Validation Precision: 0.0000, Validation Recall: 0.0000, Validation F1: 0.0000, Validation Support: 2013
Fear: Val

Epoch 2/19 => Train Loss: 1.9024, Train Accuracy: 0.2241
Angry: Train Precision: 0.1950, Train Recall: 0.1971, Train F1: 0.1961, Train Support: 15998
Disgust: Train Precision: 0.2132, Train Recall: 0.1255, Train F1: 0.1580, Train Support: 15972
Fear: Train Precision: 0.1728, Train Recall: 0.0395, Train F1: 0.0643, Train Support: 15964
Happy: Train Precision: 0.2499, Train Recall: 0.1952, Train F1: 0.2192, Train Support: 16090
Neutral: Train Precision: 0.1865, Train Recall: 0.1174, Train F1: 0.1441, Train Support: 15946
Sad: Train Precision: 0.1921, Train Recall: 0.3098, Train F1: 0.2371, Train Support: 16097
Surprise: Train Precision: 0.2723, Train Recall: 0.5844, Train F1: 0.3715, Train Support: 15933
Validation Loss: 1.8549, Validation Accuracy: 0.2861
Angry: Validation Precision: 0.2169, Validation Recall: 0.3210, Validation F1: 0.2589, Validation Support: 2003
Disgust: Validation Precision: 0.3018, Validation Recall: 0.2724, Validation F1: 0.2863, Validation Support: 1935
Fear: Val

...

Epoch 27/28 => Train Loss: 1.4100, Train Accuracy: 0.7541
Angry: Train Precision: 0.7068, Train Recall: 0.6893, Train F1: 0.6979, Train Support: 24205
Disgust: Train Precision: 0.9453, Train Recall: 0.9421, Train F1: 0.9437, Train Support: 23883
Fear: Train Precision: 0.7001, Train Recall: 0.5816, Train F1: 0.6354, Train Support: 23613
Happy: Train Precision: 0.7878, Train Recall: 0.8161, Train F1: 0.8017, Train Support: 23894
Neutral: Train Precision: 0.6630, Train Recall: 0.7222, Train F1: 0.6913, Train Support: 24106
Sad: Train Precision: 0.6560, Train Recall: 0.6720, Train F1: 0.6639, Train Support: 24011
Surprise: Train Precision: 0.8168, Train Recall: 0.8531, Train F1: 0.8345, Train Support: 24288
Validation Loss: 1.3832, Validation Accuracy: 0.7818
Angry: Validation Precision: 0.7413, Validation Recall: 0.7371, Validation F1: 0.7392, Validation Support: 2990
Disgust: Validation Precision: 0.9720, Validation Recall: 0.9584, Validation F1: 0.9651, Validation Support: 3004
Fear: Va

Epoch 28/28 => Train Loss: 1.4029, Train Accuracy: 0.7614
Angry: Train Precision: 0.7108, Train Recall: 0.7055, Train F1: 0.7081, Train Support: 23824
Disgust: Train Precision: 0.9500, Train Recall: 0.9452, Train F1: 0.9476, Train Support: 24127
Fear: Train Precision: 0.7152, Train Recall: 0.5852, Train F1: 0.6437, Train Support: 24043
Happy: Train Precision: 0.7947, Train Recall: 0.8180, Train F1: 0.8062, Train Support: 23652
Neutral: Train Precision: 0.6675, Train Recall: 0.7297, Train F1: 0.6972, Train Support: 24085
Sad: Train Precision: 0.6700, Train Recall: 0.6865, Train F1: 0.6781, Train Support: 24057
Surprise: Train Precision: 0.8205, Train Recall: 0.8589, Train F1: 0.8392, Train Support: 24212
Validation Loss: 1.3808, Validation Accuracy: 0.7850
Angry: Validation Precision: 0.7356, Validation Recall: 0.7282, Validation F1: 0.7319, Validation Support: 2954
Disgust: Validation Precision: 0.9479, Validation Recall: 0.9555, Validation F1: 0.9517, Validation Support: 3008
Fear: Va

TEST ACCURACY_:  0.41517857142857145
TEST ACCURACY:  0.7815238095238095


test_accuracy,▁
test_accuracy_,▁
train_accuracy,▁
train_f1_Angry,▁
train_f1_Disgust,▁
train_f1_Fear,▁
train_f1_Happy,▁
train_f1_Neutral,▁
train_f1_Sad,▁
train_f1_Surprise,▁
train_loss,▁


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.


**Live emotion detection**

In [ ]:
num_classes = 7
your_label_mapping = {0: 'Angry', 1: 'Disgust', 2: 'Fear', 3: 'Happy', 4: 'Neutral', 5: 'Sad', 6: 'Surprise'}
model = EmotionCNN(num_classes)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.load_state_dict(torch.load('../Models/emotion_recognition_model.pth', map_location=device))
model.to(device)
model.eval() 

# initialize the face detector
detector = dlib.get_frontal_face_detector()

# initialize the camera
cap = cv2.VideoCapture(0)

# apply the transformations to the face image
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((48, 48)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

while True:
    # read a frame from the camera
    ret, frame = cap.read()

    # faces detection
    faces = detector(frame)

    # if there is at least one face detected, process the image
    if len(faces) > 0:
        # take only the first face
        face = faces[0]
        
        # cut the face from the frame
        x, y, w, h = face.left(), face.top(), face.width(), face.height()
        face_image = frame[y:y+h, x:x+w]

        # check if the face image is not empty
        if not face_image.size == 0:
            # apply the transformations to the face image
            pil_image = Image.fromarray(cv2.cvtColor(face_image, cv2.COLOR_BGR2RGB))
            input_image = transform(pil_image).unsqueeze(0)  # Aggiunge una dimensione di batch
            input_image = input_image.to(device)

            # model prediction
            with torch.no_grad():
                output = model(input_image)

            # get the label predicted by the model
            _, predicted = torch.max(output, 1)
            predicted_emotion = your_label_mapping[predicted.item()]

            print(f'Predicted Emotion: {predicted_emotion}')

    # show the frame with the face rectangle added
    cv2.imshow("Face Detection", frame)

    # wait for 2 seconds (time in milliseconds)
    cv2.waitKey(1000)

    # if q is pressed, terminate the loop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# release the capture
cap.release()
cv2.destroyAllWindows()